# Parallelism Benchmarks
This notebook illustrates the effect of enabling scheduler parallel execution across backends using real simulators.

Figures are saved to `../figures/` and tables to `../results/`.


In [ ]:
from pathlib import Path
FIGURES_DIR = Path('../figures')
RESULTS_DIR = Path('../results')
FIGURES_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)
try:
    import ipynbname
    NB_NAME = ipynbname.path().stem
except Exception:  # pragma: no cover
    NB_NAME = 'notebook'


In [ ]:

from quasar import Circuit, Scheduler
from quasar.cost import Backend
from quasar.backends import StimBackend, MPSBackend
from quasar.planner import Planner
from benchmarks.runner import BenchmarkRunner
import quasar.config as config
import pandas as pd
import matplotlib.pyplot as plt

config.DEFAULT.dd_metric_threshold = 999  # disable decision-diagram backend

runner = BenchmarkRunner()


def stim_circuit(reps: int = 500) -> Circuit:
    ops = []
    for _ in range(reps):
        ops.extend([
            {"gate": "H", "qubits": [0]},
            {"gate": "H", "qubits": [1]},
            {"gate": "CX", "qubits": [0, 1]},
            {"gate": "H", "qubits": [2]},
            {"gate": "H", "qubits": [3]},
            {"gate": "CX", "qubits": [2, 3]},
        ])
    return Circuit(ops)


def mps_circuit(reps: int = 100) -> Circuit:
    ops = []
    for _ in range(reps):
        ops.extend([
            {"gate": "H", "qubits": [0]},
            {"gate": "T", "qubits": [0]},
            {"gate": "CX", "qubits": [0, 1]},
            {"gate": "H", "qubits": [2]},
            {"gate": "T", "qubits": [2]},
            {"gate": "CX", "qubits": [2, 3]},
        ])
    return Circuit(ops)

stim = stim_circuit()
mps = mps_circuit()

# Stim benchmarks
scheduler_stim_parallel = Scheduler(
    backends={Backend.TABLEAU: StimBackend()},
    planner=Planner(backend_order=[Backend.TABLEAU]),
    parallel_backends=[Backend.TABLEAU],
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
)

scheduler_stim_serial = Scheduler(
    backends={Backend.TABLEAU: StimBackend()},
    planner=Planner(backend_order=[Backend.TABLEAU]),
    parallel_backends=[],
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
)

res_stim_parallel = runner.run_quasar(stim, scheduler_stim_parallel, backend=Backend.TABLEAU)
res_stim_serial = runner.run_quasar(stim, scheduler_stim_serial, backend=Backend.TABLEAU)

# MPS benchmarks
scheduler_mps_parallel = Scheduler(
    backends={Backend.MPS: MPSBackend()},
    planner=Planner(backend_order=[Backend.MPS]),
    parallel_backends=[Backend.MPS],
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
)

scheduler_mps_serial = Scheduler(
    backends={Backend.MPS: MPSBackend()},
    planner=Planner(backend_order=[Backend.MPS]),
    parallel_backends=[],
    quick_max_qubits=None,
    quick_max_gates=None,
    quick_max_depth=None,
)

res_mps_parallel = runner.run_quasar(mps, scheduler_mps_parallel, backend=Backend.MPS)
res_mps_serial = runner.run_quasar(mps, scheduler_mps_serial, backend=Backend.MPS)


df = pd.DataFrame([
    {"backend": "Stim", "mode": "serial", "runtime": res_stim_serial["run_time"], "memory": res_stim_serial["run_peak_memory"]},
    {"backend": "Stim", "mode": "parallel", "runtime": res_stim_parallel["run_time"], "memory": res_stim_parallel["run_peak_memory"]},
    {"backend": "MPS", "mode": "serial", "runtime": res_mps_serial["run_time"], "memory": res_mps_serial["run_peak_memory"]},
    {"backend": "MPS", "mode": "parallel", "runtime": res_mps_parallel["run_time"], "memory": res_mps_parallel["run_peak_memory"]},
])

df


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
df.pivot(index="backend", columns="mode", values="runtime").plot.bar(ax=axes[0])
axes[0].set_ylabel("Runtime (s)")
df.pivot(index="backend", columns="mode", values="memory").plot.bar(ax=axes[1])
axes[1].set_ylabel("Peak memory (bytes)")
plt.tight_layout()


In [ ]:

stim_speedup = res_stim_serial["run_time"] / res_stim_parallel["run_time"] if res_stim_parallel["run_time"] > 0 else float('inf')
mps_speedup = res_mps_serial["run_time"] / res_mps_parallel["run_time"] if res_mps_parallel["run_time"] > 0 else float('inf')
print(f"Stim parallel {res_stim_parallel['run_time']:.3f}s vs serial {res_stim_serial['run_time']:.3f}s -> {stim_speedup:.2f}x speedup")
print(f"Stim memory parallel {res_stim_parallel['run_peak_memory']}B vs serial {res_stim_serial['run_peak_memory']}B")
print(f"MPS parallel {res_mps_parallel['run_time']:.3f}s vs serial {res_mps_serial['run_time']:.3f}s -> {mps_speedup:.2f}x speedup")
print(f"MPS memory parallel {res_mps_parallel['run_peak_memory']}B vs serial {res_mps_serial['run_peak_memory']}B")
results = df.to_dict(orient='records')


In [ ]:

# Record parameters and results
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'notebook'

# Collect simple parameters from globals
_params = {
    k: v for k, v in globals().items()
    if not k.startswith('_') and isinstance(v, (int, float, str, bool, list, dict, tuple))
}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f"../results/{nb_name}_params.json", 'w') as f:
    json.dump(_params, f, indent=2, default=str)
if 'results' in globals():
    try:
        with open(f"../results/{nb_name}_results.json", 'w') as f:
            json.dump(results, f, indent=2)
    except TypeError:
        pass
print(json.dumps(_params, indent=2, default=str))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
for i, num in enumerate(plt.get_fignums(), start=1):
    plt.figure(num)
    plt.savefig(FIGURES_DIR / f'{NB_NAME}_plot{i}.png')
for _name, _obj in globals().items():
    if isinstance(_obj, pd.DataFrame):
        _obj.to_csv(RESULTS_DIR / f'{NB_NAME}_{_name}.csv', index=False)
        try:
            _obj.to_latex(RESULTS_DIR / f'{NB_NAME}_{_name}.tex', index=False)
        except Exception:
            pass
